In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os
from pydataset import data
from math import sqrt
from scipy import stats

#Acquire functions
from acquire import get_connection

#Prepare function
#from prepare import 

#sklearn for machine learning
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
#from sklearn.tree import export_graphviz
import sklearn.metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import env
import explore
import prepare
from prepare import prep_telco

#import graphviz
#from graphviz import Graph

**Telco Churn Classification Project**

Project Planning

**Executive Summary - Conclusions & Next Steps**
I found that all of the classification models I created DecisionTree, RandomForest, and KNeighborsthey all predicted the churn of customers with similar accuracy with KNN performing the best, using the features gender, contract_type, partner, dependents, phone_service,multiple_lines, internet_service_type and payment_type.
I chose my KNN model as my best model with a 78% accuracy rate for predicting my target value, churn. This model outperformed my baseline score of 73% accuracy, so it has value.
Some initial exploration and statistical testing revealed that low tenure can be linked to high churn, senior citizens churn more often than non-seniors and fiber optics subscribers churn much more than any other internet type subscriber so these would be the area I would focus on with more time that might increase the model predictions toward a greater accuracy.

Takeaways from heatmaps -month-to-month customers churn the most of contract_type approximately 43% -Fiber optics has the highest rate of churn at 43% churning of this internet_service_type -Electronic check has the highest churn  at 46% -senior_citizens churn at a much higher rate than other customers 44% compared to 23% for overall customers

Countplot Takeaways -tenure is by far highest at one month and by far least at 72 months -customers with the lowest monthly bills are churning the most -fiber optics service type churns at a rate by far higher than the other two -those customers without partners churn at a much higher rate -customers with dependents churn at a slightly higher rate -seniors churn at a much higher rate than other customers -customers who pay with electronic check churn near as much as all the other 3 payment types combined

Takeaway from pairplot -customers churn more often at the lower tenures -customers churn the lesast with high monthly bills and high tenure -senior citizens with all types of tenure churn at a similar rate unlike the rest of the population that mostly churns at a lower tenure

Violinplot takeaways -churn is much higher at the lowest tenure -churn is higher for customers with higher monthly charges

   **Acquire**

In [2]:
#acquire the data

df = pd.read_sql('''SELECT *
FROM customers
JOIN contract_types USING(contract_type_id)
JOIN internet_service_types USING(internet_service_type_id)
JOIN payment_types USING(payment_type_id);''', get_connection("telco_churn"))

In [3]:
#check it out
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [4]:
#get the shape
df.shape

(7043, 24)

In [5]:
#return a summary of statistics
df.describe()

,payment_type_id,internet_service_type_id,contract_type_id,senior_citizen,tenure,monthly_charges
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,2.315633,1.872923,1.690473,0.162147,32.371149,64.761692
std,1.148907,0.737796,0.833755,0.368612,24.559481,30.090047
min,1.000000,1.000000,1.000000,0.000000,0.000000,18.250000
25%,1.000000,1.000000,1.000000,0.000000,9.000000,35.500000
50%,2.000000,2.000000,1.000000,0.000000,29.000000,70.350000
75%,3.000000,2.000000,2.000000,0.000000,55.000000,89.850000
max,4.000000,3.000000,3.000000,1.000000,72.000000,118.750000


**Acquire Takeaways**
I wrote a sql query to acquire my data from the Codeup database.
I'm importing my get_connection function from my acquire.py file 
I'm bringing in 17043 rows and 24 columns on my initial acquisition.

**Prepare**

In [6]:
# This function takes in a dataframe and preps it according to my specifications
prep_telco(df)

In [7]:
# example:          train, validate, test = prepare.prep_iris(df)

In [9]:
#define the target
target='churn'

In [10]:
# This function takes in a dataframe, and returns he function returns, in this order, train, 
# validate and test dataframes. 
prepare.train_validate_test_split(df, target, seed=123)

(      gender  senior_citizen partner dependents  tenure phone_service  \
 6096  Female               0     Yes         No      70           Yes   
 1603    Male               0     Yes        Yes      15           Yes   
 5350  Female               1     Yes         No      52           Yes   
 2068    Male               0      No         No      39            No   
 6366  Female               0     Yes         No      32           Yes   
 ...      ...             ...     ...        ...     ...           ...   
 296   Female               0     Yes        Yes      71           Yes   
 4200    Male               1     Yes         No      16           Yes   
 3012    Male               0     Yes         No      40           Yes   
 763   Female               0      No         No      11           Yes   
 3935  Female               0     Yes        Yes      10           Yes   
 
         multiple_lines paperless_billing  monthly_charges total_charges churn  \
 6096                No     

In [13]:
prepare.split_data(df)

(      gender  senior_citizen partner dependents  tenure phone_service  \
 6096  Female               0     Yes         No      70           Yes   
 1603    Male               0     Yes        Yes      15           Yes   
 5350  Female               1     Yes         No      52           Yes   
 2068    Male               0      No         No      39            No   
 6366  Female               0     Yes         No      32           Yes   
 ...      ...             ...     ...        ...     ...           ...   
 296   Female               0     Yes        Yes      71           Yes   
 4200    Male               1     Yes         No      16           Yes   
 3012    Male               0     Yes         No      40           Yes   
 763   Female               0      No         No      11           Yes   
 3935  Female               0     Yes        Yes      10           Yes   
 
         multiple_lines paperless_billing  monthly_charges total_charges churn  \
 6096                No     

In [15]:
print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

NameError: name 'train' is not defined

**Prepare Takeaways**
I checked for Null values; this is a squeaky clean dataset, so I don't have to deal with missing values.
The target classes are perfectly balanced with 50 of each species, so I don't have to worry about unbalanced data.
On initial viewing of distributions of features, I can see that while sepal length and width are relatively normally distributed, petal length and width have a bimodal distribution which may be of value in identifying species, the target. I'll look at how species plays into these bimodal distributions in explore, after I split my data.
species_id is not a useful column to me going forward, so I will drop it before I split.
As my numeric columns are in the same unit of measure and range of values reasonible, I won't need to scale my numeric columns before modeling.
I'll split my data into train, validate, and test datasets, stratifying my species column.